## Analysis of Live Streaming Health Metrics (Using Twitch Interaction Data)

### Project Introduction

This report conducts an Exploratory Data Analysis (EDA) on a Twitch live-streaming interaction dataset. The primary goal is to understand user and streamer behavior to inform the design of a "health metric" framework for live-streaming platforms.

### Objectives:

1. To process and analyze a large dataset using a combination of Pandas and PySpark.

2. To calculate key metrics related to platform activity, user engagement, and content supply.

3. To visualize trends and distributions to uncover initial insights.

### Dataset:
The dataset contains a sample of 100,000 users' viewing interactions on Twitch. The columns are:

- UserId: Unique identifier for a viewer.

- StreamID: Unique identifier for a specific live stream segment.

- Streamer: The name of the streamer.

- StartTime: The time (in minutes from a relative start) the user began watching.

- StopTime: The time (in minutes) the user stopped watching.

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "100k_a.csv"
column_names = ['UserId', 'StreamID', 'Streamer', 'StartTime', 'StopTime']

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "volodymyrpivoshenko/twitch-live-streaming-interactions-sample-dataset",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
  pandas_kwargs={"header": None, "names": column_names}
)

### 2. Data Preparation and PySpark Initialization

The data is initially loaded into a Pandas DataFrame for convenience. To handle potentially larger datasets and leverage distributed computing capabilities, we will convert it into a PySpark DataFrame. This section initializes the Spark session and prepares the data for analysis.

In [ ]:
# data check
df.shape
df.head()
!java -version
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Twitch Live Health Analysis").getOrCreate()
spark_df = spark.createDataFrame(df)
spark_df.printSchema()
spark_df.show(5)

### 3. Data Preprocessing and Quality Assessment

Before diving into analysis, it's crucial to assess the quality of the data. We will check for correct data types (schema) and look for any missing or null values.

In [ ]:
from pyspark.sql.functions import col, isnan, when, count

def data_type_diagnostics(df):
    print("===== Data type（Schema）=====")
    print(df.dtypes)
    print("\n")

data_type_diagnostics(spark_df)

In [ ]:
def data_quality_diagnostics(df):
    exprs = []
    for c, t in df.dtypes:
        if t in ('bigint', 'int', 'double', 'float'):
            exprs.append(count(when(col(c).isNull() | isnan(col(c)), c)).alias(c))
        elif t == 'string':
            exprs.append(count(when(col(c).isNull() | (col(c) == ""), c)).alias(c))
    df.select(exprs).show()

data_quality_diagnostics(spark_df)

#### Finding:
The data quality check reveals that the dataset is remarkably clean. All columns have appropriate data types, and there are zero null or empty values. This allows us to proceed directly to the analysis phase without needing extensive data cleaning.

### 4. Exploratory Data Analysis (EDA)

In this section, we will compute various metrics to understand the platform's overall scale, user engagement patterns, and content dynamics.

### 4.1. High-Level Platform Metrics

We begin by calculating fundamental metrics to understand the size and scope of the platform based on this dataset. This includes the number of unique users, streamers, and stream segments.

In [ ]:
from pyspark.sql.functions import avg, countDistinct, explode, max as spark_max, sequence, col

# Basic statistics
def basic_statistics(df):
    # Number of users
    user_count = df.select("UserId").distinct().count()
    print(f"Unique User Count: {user_count}")

    # Number of streamers
    streamer_count = df.select("Streamer").distinct().count()
    print(f"Unique Streamer Count: {streamer_count}")

    # Total number of stream segments
    total_streams = df.select("StreamID").distinct().count()
    print(f"Total StreamID Count: {total_streams}")

    # Average number of segments per streamer
    avg_streams_per_streamer = df.groupBy("Streamer").count().agg(avg("count"))
    avg_streams_per_streamer.show()

    # Number of viewers per StreamID
    watchers_per_stream = df.groupBy("StreamID").agg(countDistinct("UserId").alias("WatcherCount"))
    watchers_per_stream.select(avg("WatcherCount").alias("AvgWatcherCount")).show()

    # Streamers with the highest number of concurrent viewers in their streams
    # 1. Expand each user's time slots in the stream segment
    df_expanded = df.withColumn(
        "TimeSlot",
        explode(sequence(col("StartTime"), col("StopTime") - 1))
    )

    # 2. Aggregate user count by StreamID + TimeSlot
    users_per_slot = df_expanded.groupBy("StreamID", "TimeSlot") \
        .agg(countDistinct("UserId").alias("UsersCount"))

    # 3. Get the maximum user count for each StreamID
    max_concurrent_users = users_per_slot.groupBy("StreamID") \
        .agg(spark_max("UsersCount").alias("MaxConcurrentUsers"))

    # 4. Show results
    max_concurrent_users.orderBy("MaxConcurrentUsers", ascending=False).show(10)

    # Match StreamID to Streamer, take the maximum and deduplicate by streamer
    max_concurrent_users_with_streamer = max_concurrent_users.join(
        df.select("StreamID", "Streamer").distinct(),
        on="StreamID",
        how="left"
    )
    max_concurrent_users_with_streamer = max_concurrent_users_with_streamer.dropDuplicates(["Streamer"]).orderBy("MaxConcurrentUsers", ascending=False)
    max_concurrent_users_with_streamer.show(10)

basic_statistics(spark_df)

#### Summary of High-Level Metrics:

- Unique Users: 100,000

- Unique Streamers: 162,625. This is a fascinating insight: in this sample, there are more streamers than unique viewers, suggesting a very large supply of content creators relative to the viewer base.

- Total Stream Segments (StreamIDs): 739,991

- Average Streams per Streamer: ~18.8

- Average Watchers per Stream: ~4.1. This low number suggests that the vast majority of streams have a very small audience, pointing towards a long-tail distribution of popularity.

- Max Concurrent Users: The analysis shows that top streamers like pokemon, cloud9, and nintendo can attract a significant number of concurrent viewers (over 150-200), standing in sharp contrast to the average.

---

### 4.2. User Behavior Metrics (Watch Duration & Time Analysis)

Next, we analyze how users engage with content, focusing on the duration of their viewing sessions. We also convert the relative timestamps into actual datetime objects to enable time-series analysis.

In [ ]:
# Viewing duration distribution
from pyspark.sql.functions import (avg, max as spark_max, min as spark_min)

df_with_duration = spark_df.withColumn("WatchDuration", col("StopTime") - col("StartTime"))
df_with_duration.select(
    avg("WatchDuration").alias("AvgWatchDuration"),
    spark_min("WatchDuration").alias("MinWatchDuration"),
    spark_max("WatchDuration").alias("MaxWatchDuration")
).show()

#### Finding:
The average watch duration is only 3.14 minutes (since StartTime and StopTime are in minutes). This suggests that many interactions are very short, possibly representing users "channel surfing" or quickly checking out different streams.

In [ ]:
# Since there is no given start date, assume it begins on August 1, 2025
# Start date conversion
from pyspark.sql.functions import expr
base_date = "2025-08-01 00:00:00"

df_with_datetime = spark_df.withColumn(
    "StartDateTime",
    expr(f"timestamp('{base_date}') + INTERVAL 1 MINUTES * StartTime")
).withColumn(
    "StopDateTime",
    expr(f"timestamp('{base_date}') + INTERVAL 1 MINUTES * StartTime")
)

# or expr(f"timestamp('{base_date}') + make_interval(0, 0, 0, 0, 0, StartTime*10, 0)")

df_with_datetime.select("UserId", "Streamer", "StartTime", "StartDateTime", "StopDateTime").show(5)

### Data Assumption:
Since the dataset provides relative time in minutes, a base date of 2025-08-01 has been assumed to facilitate time-series analysis. This is an artificial construct necessary for this analysis, and any conclusions about specific dates should be viewed with this in mind.

In [ ]:
# Daily statistics of active users, number of broadcasts, and platform peak concurrent users
from pyspark.sql.functions import to_date
df_with_day = df_with_datetime.withColumn(
    "Day",
    to_date(col("StartDateTime"))
)
df_with_day.show(10)

daily_active_users_count = df_with_day.groupBy("Day").agg(countDistinct("UserId").alias("ActiveUserCount"))

daily_active_streamer_count = df_with_day.groupBy("Day").agg(countDistinct("Streamer").alias("ActiveStreamerCount"))

df_with_day_expanded = df_with_day.withColumn("TimeSlot", explode(sequence(col("StartTime"), col("StopTime") - 1)))
df_with_day_expanded = df_with_day_expanded.withColumn("SlotDatetime", expr(f"timestamp('{base_date}') + INTERVAL 1 MINUTES * TimeSlot"))
users_per_slot = df_with_day_expanded.groupBy("Day", "SlotDatetime").agg(countDistinct("UserId").alias("UsersCountPerSlot"))
daily_max_concurrent_users = users_per_slot.groupBy("Day").agg(spark_max("UsersCountPerSlot").alias("MaxConcurrentUsers"))

daily_summary_per_slot = daily_active_users_count.join(daily_active_streamer_count, on='Day', how='left').join(users_per_slot, on='Day', how='left').orderBy("UsersCountPerSlot")
daily_summary_per_day = daily_active_streamer_count.join(daily_active_users_count, on='Day', how='left').join(daily_max_concurrent_users, on='Day', how='left') 

daily_summary_per_slot.show(10)
daily_summary_per_day.show(10)


### 4.3. Visual Analysis

Visualizations are essential for understanding trends and distributions. We will now plot the metrics calculated above.

#### Daily Activity Trends

This plot shows the daily active users, active streamers, and the platform's peak concurrent users over the analysis period.

In [ ]:
import matplotlib.pyplot as plt

daily_summary_per_day_pd = daily_summary_per_day.toPandas()
daily_summary_per_day_pd = daily_summary_per_day_pd.sort_values("Day") 

fig, ax1 = plt.subplots(figsize=(12,6))

ax2 = ax1.twinx() 

ax1.plot(daily_summary_per_day_pd['Day'], daily_summary_per_day_pd['ActiveUserCount'], marker='o', label='Active Users', color='blue')
ax1.plot(daily_summary_per_day_pd['Day'], daily_summary_per_day_pd['ActiveStreamerCount'], marker='o', label='Active Streamers', color='green')

ax2.plot(daily_summary_per_day_pd['Day'], daily_summary_per_day_pd['MaxConcurrentUsers'], marker='o', label='Max Concurrent Users', color='red')

ax1.set_xlabel("Day")
ax1.set_ylabel("Users/Streamers")
ax2.set_ylabel("Max Concurrent Users")

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
plt.title("Daily Twitch Activity Metrics")
plt.xticks(rotation=45)
plt.show()

#### Insight:
The plot reveals daily fluctuations in platform activity. The number of active users and streamers generally move in tandem. The peak concurrent users (red line) show a slightly different pattern, indicating that overall activity doesn't always translate to higher peak concentration. A longer time series would be needed to confirm if a weekly cyclical pattern (e.g., weekend peaks) exists.

---

#### Distribution of Watch Durations
This histogram shows the frequency of different watch durations. A logarithmic scale is used on the y-axis to better visualize the long-tail nature of the data.

In [ ]:
df_with_duration = spark_df.withColumn("WatchDuration", col("StopTime") - col("StartTime"))
duration_pd = df_with_duration.select("WatchDuration").toPandas()

plt.figure(figsize=(8,5))
plt.hist(duration_pd["WatchDuration"], bins=50, color="skyblue", edgecolor="black")
plt.xlabel("Watch Duration (10-min units)")
plt.ylabel("Frequency")
plt.title("Distribution of Watch Durations")
plt.yscale("log") 

#### Insight:
The histogram is heavily skewed to the right, confirming the low average watch duration found earlier. The logarithmic scale highlights a classic long-tail distribution: a massive number of very short interactions (1-2 minutes) and a rapidly decreasing number of longer viewing sessions. This is typical user behavior on content discovery platforms.

---

#### Distribution of Watchers per Stream
This histogram shows how many streams have a certain number of unique viewers. Again, a log scale is used.

In [ ]:
watchers_per_stream = spark_df.groupBy("StreamID").agg(countDistinct("UserId").alias("WatcherCount"))
watchers_pd = watchers_per_stream.toPandas()

plt.figure(figsize=(8,5))
plt.hist(watchers_pd["WatcherCount"], bins=50, color="orange", edgecolor="black")
plt.xlabel("Unique Watchers per Stream")
plt.ylabel("Frequency")
plt.title("Distribution of Watchers per StreamID")
plt.yscale("log") 
plt.show()

#### Insight:
Similar to watch duration, the number of unique watchers per stream also follows a long-tail pattern. The vast majority of streams have only one or two viewers. This reinforces the "average watchers" metric and shows that only a small fraction of streams become popular, while most content has a very niche audience.

#### Streamer Activity vs. Popularity
This scatter plot explores the relationship between how frequently a streamer broadcasts (Stream Count) and how many unique viewers they attract across all their streams. Both axes are logarithmic to better handle the wide range of values.

In [ ]:
streamer_stats = spark_df.groupBy("Streamer") \
    .agg(countDistinct("StreamID").alias("StreamCount"),
         countDistinct("UserId").alias("UniqueWatchers"))

streamer_pd = streamer_stats.toPandas()

plt.figure(figsize=(8,6))
plt.scatter(streamer_pd["StreamCount"], streamer_pd["UniqueWatchers"], alpha=0.5)
plt.xlabel("Number of Streams per Streamer")
plt.ylabel("Unique Watchers")
plt.title("Streamer Stream Count vs Unique Watchers")
plt.xscale("log")
plt.yscale("log")
plt.show()

In [ ]:
user_activity = spark_df.groupBy("UserId").count().withColumnRenamed("count", "InteractionCount")
user_pd = user_activity.toPandas()

plt.figure(figsize=(8,5))
plt.hist(user_pd["InteractionCount"], bins=50, color="purple", edgecolor="black")
plt.xlabel("Interactions per User")
plt.ylabel("Number of Users")
plt.title("User Activity Distribution")
plt.yscale("log")
plt.show()

#### Insight:
The plot shows a clear positive correlation: streamers who broadcast more tend to attract a larger unique audience. However, the relationship is not strictly linear. For any given stream count, there is a wide variance in the number of unique watchers. This suggests that while consistency (higher stream count) is important, it is not the only factor for success. Content quality, streamer personality, or existing fame likely play a significant role, allowing some streamers to achieve high viewership with relatively few streams.

### 5. Conclusion and Next Steps

This initial EDA has provided several key insights into the platform's dynamics:

- High Content Supply: The number of streamers is very high relative to the sampled user base.

- Long-Tail Engagement: User engagement, both in terms of watch duration and stream popularity, follows a strong long-tail distribution. Most interactions are short, and most streams have few viewers.

- Activity Correlates with Popularity: There is a positive, though not deterministic, relationship between the number of times a streamer goes live and the size of their audience.

### Next Steps for a Deeper Analysis:

- User/Streamer Segmentation: Classify users (e.g., casual vs. power users) and streamers (e.g., top-tier, mid-tier, long-tail) to analyze their behaviors separately.

- Retention Analysis: Although difficult without explicit dates, a cohort analysis could be performed on the relative timeline to understand user retention, a key health metric.

- Time-of-Day Analysis: With a corrected StopDateTime, analyze peak viewing hours to understand platform usage patterns better.

- Refine Health Metrics: Based on these findings, a more formal "health framework" could be proposed, incorporating metrics for Content Richness (streamer count), User Engagement (DAU, watch duration distribution), and Content Discovery (how well the platform surfaces long-tail content).